# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [50]:
%reload_ext autoreload
%autoreload 2
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio
# import importlib
# importlib.reload(VectorDatabase)
# importlib.reload(TextFileLoader)
# importlib.reload(CharacterTextSplitter)

In [51]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [52]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)

1

In [53]:
print(documents[0][:100])

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [54]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

105

Let's take a look at some of the documents we've managed to split.

In [55]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [56]:
import os
import openai
from getpass import getpass
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key
#print("API Key: ", os.environ["OPENAI_API_KEY"])

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes per API Docs 
You are receiving a specific length of the embedding vector based on the model you choose, they chose a wrapper around model with 1536 return dimensions (text-embedding-3-small), but you can choose others when choosing a model.
Choosing text-embedding-3-large will give you 3072 by default.
To reduce the embedding's dimensions without losing its concept-representing properties, pass in the dimensions parameter to 
the POST /embeddings endpoint, its an optional integer parameter

https://platform.openai.com/docs/guides/embeddings

2. Matryoshka Representation Learning (MRL) which encodes information at different granularities and allows a single embedding to adapt to the computational constraints of downstream tasks.
Matryoshka dolls, or Russian nesting dolls, are a set of decreasingly sized wooden dolls placed one inside another, symbolizing motherhood, fertility, and family
The Matryoshka Representations improve efficiency for large-scale classification and retrieval
without any significant loss of accuracy.   using a coarse to fine direction.





We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [57]:
# Improvement 3 change the embedding model to use a different model
from aimakerspace.openai_utils.embedding import EmbeddingModel
# importlib.reload(EmbeddingModel)
embedding_model_algorithm = "text-embedding-3-small"#"text-embedding-3-large"#"text-embedding-ada-002"#"text-embedding-3-small"
embedding_model = EmbeddingModel(embedding_model_algorithm)

vector_db = VectorDatabase(embedding_model)
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Async will allow us to dispatch jobs without waiting for them to complete.  This allows us to call abuild_from_list and not wait for any particular job to finish.  These return inPython a Coroutine object aka a Promise or a Future.  Its just a handle to be referenced or await'ed or .then()'d if in typescript.  
since Jupyter is all sync IO we have to add an async runner to create an event loop, run the async function to completeion then return the result, else we'd endup with a Promise and not the result of the call.  

The reason we should bother with the async is eventually EmbeddingModel.async_get_embeddings gets called wtih an array of text chunks, and each of these needs to go to the OpenAI API.  Since wer'e dispatching a lot of API calls, we should do them concurrently to get a result much faster than if we launched them one by one.  You need pretty good 'what if something fails' semantics in here so a single 500 error or if you hit a rate limit things keep working.

a synchronous job will run one after another so again we'd wait for a long time doing 1 by 1 encoding.



So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [58]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
* Pinning your production applications to specific model snapshots (like gpt-5-2025-08-07 for example) to ensure consistent behavior
* Building evals that measure the behavior of your prompts so you can monitor prompt performance as you iterate, or when you change and upgrade model versions
* Reusable prompts could also ensure you always use the same prompts.  Reduces variablility due to whitespace, message order, etc..
* if an api supports 'greedy decoding' that will be determnistic.
* Normally I'd say to set the temperature really high and top-p really low as well, so you're essentially reducing the possibility there are multiple options for the LLM to choose from and its not introducing any variablity with the temperature.

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [59]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI
# importlib.reload(ChatOpenAI)
# importlib.reload(UserRolePrompt)
# importlib.reload(SystemRolePrompt)
# importlib.reload(AssistantRolePrompt)

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [60]:
print(response)

Hello! The "best" way to write a loop in Python really depends on what you're trying to accomplish. Generally, Python offers a couple of common loop structures:

1. **For loops** – great for iterating over sequences (like lists, strings, ranges):
```python
for item in sequence:
    # do something with item
```

2. **While loops** – useful when you want to loop until a certain condition is met:
```python
while condition:
    # do something
```

Here's a quick tip: When looping over a known sequence or range, a `for` loop is usually cleaner and more Pythonic. For example, to print numbers 0 through 4:

```python
for i in range(5):
    print(i)
```

If you need to loop until a condition changes dynamically, use a `while` loop:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

Also, remember to keep loops simple and avoid unnecessary complexity for readability.

If you have a specific task in mind, feel free to share, and I can help craft the best loop for that! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [61]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [62]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False, distance_measure_string: str = "cosine_similarity") -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores
        self.distance_measure_string = distance_measure_string
        
    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k, distance_measure_string= self.distance_measure_string)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [63]:
import datetime as datetime
import pandas as pd
import os as os
print(os.getcwd())
# %pip install vscodedebugvisualizer
# Addition #4 add list of distance_measures to test here
distance_measure_options = ["cosine_similarity", "euclidean_distance", "manhattan_distance", "chebyshev_distance"]
distance_measure_string = "euclidean_distance"

rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True,
    distance_measure_string=distance_measure_string
)

QA_MODE = True
COMPARE_TO_BASELINE = True
PERFORM_NEW_RUN = True
BASELINE_FILE = "results.experiment_baseline.csv"
COMPARE_TO_FILE = "results.experiment_baseline.csv"

if QA_MODE:
    test_folder = "test"
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)
    if PERFORM_NEW_RUN:
        dynamic_filename_part = "experiment_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"results.{dynamic_filename_part}.csv"
        results = []
        # here is the csv header.. dont need to add since pandas will do it later
        header = {
                "question_number": "question_number",
                "question": "question",
                "response": "response",
                "context_count": "context_count",
                "similarity_score_1": "similarity_score_1",
                "similarity_score_2": "similarity_score_2",
                "similarity_score_3": "similarity_score_3"
            }
        
        results_df = pd.DataFrame(results)
        question_counter = 1
        question_array = [
            "What are the early warning signs of heart disease and when should I see a doctor?",
            "What lifestyle changes can help manage Type 2 diabetes without medication?",
            "How is Alzheimer's disease diagnosed and what are the current treatment options?",
            "What are the most effective treatments for chronic obstructive pulmonary disease (COPD)?",
            "What screening tests are recommended for early detection of colorectal cancer?",
            "How can chronic kidney disease be slowed or prevented from progressing?",
            "What are the warning signs of a stroke and what should I do if someone is having one?",
            "What medications are commonly prescribed for managing clinical depression?",
            "How is arthritis diagnosed and what treatments help reduce joint inflammation?",
            "What are the best approaches for treating and managing obesity long-term?",
            "what is the best treatment for Celiac disease?",
            "what are the common symptoms of Lupus?"
        ]
        
        for question in question_array:
            result = rag_pipeline.run_pipeline(
                question,
                k=3,
                response_length="comprehensive", 
                include_warnings=True,
                confidence_required=True
            )
            print(f"Response: {result['response']}")
            print(f"\nContext Count: {result['context_count']}")
            print(f"Similarity Scores: {result['similarity_scores']}")
            print(f"Question {question_counter} of {len(question_array)}")
            print("--------------------------------")
            similarity_score_1_numeric = float(result['similarity_scores'][0].split(":")[1].strip())
            similarity_score_2_numeric = float(result['similarity_scores'][1].split(":")[1].strip())
            similarity_score_3_numeric = float(result['similarity_scores'][2].split(":")[1].strip())
            results.append( {
                "question_number": question_counter,
                "question": question,
                "response": result['response'],
                "context_count": result['context_count'],
                "similarity_score_1": similarity_score_1_numeric,
                "similarity_score_2": similarity_score_2_numeric,
                "similarity_score_3": similarity_score_3_numeric
            })
            question_counter += 1
        # save the results to a csv file
        results_df = pd.DataFrame(results)
        breakpoint()

        display(results_df)
        results_df.to_csv(f"./{test_folder}/{filename}", index=False)
        print(f"Results saved to {filename}")
    else:
        results_df = pd.read_csv(f"./{test_folder}/{COMPARE_TO_FILE}")
        display(results_df)
    if COMPARE_TO_BASELINE:
        baseline_results_df = pd.read_csv(f"./{test_folder}/{BASELINE_FILE}")
        # walk throubh both dataframes with baseline_results_df being the baselin
        # and results_df being the new results
        # they will have the same question number
        # compare for each question_number the response, context_count, and similarity_score_1, similarity_score_2, similarity_score_3
        # produce a new dataframe that is the comparision of the two aligned by question_number
        # save the new dataframe to a csv file
        comparison_df = pd.merge(baseline_results_df, results_df, on='question_number', suffixes=('_baseline', '_new'))
        comparison_df_with_summary = comparison_df.copy()
        comparison_df_with_summary['summary'] = comparison_df.apply(
            lambda row: 'No Change' if row['response_baseline'] == row['response_new'] and row['context_count_baseline'] == row['context_count_new'] and row['similarity_score_1_baseline'] == row['similarity_score_1_new'] and row['similarity_score_2_baseline'] == row['similarity_score_2_new'] and row['similarity_score_3_baseline'] == row['similarity_score_3_new'] else 'Change', axis=1)
        comparison_df_with_summary['similarity_score_1_diff'] = comparison_df.apply(
            lambda row: float(row['similarity_score_1_new']) - float(row['similarity_score_1_baseline']), axis=1)
        comparison_df_with_summary['similarity_score_2_diff'] = comparison_df.apply(
            lambda row: float(row['similarity_score_2_new']) - float(row['similarity_score_2_baseline']), axis=1)
        comparison_df_with_summary['similarity_score_3_diff'] = comparison_df.apply(
            lambda row: float(row['similarity_score_3_new']) - float(row['similarity_score_3_baseline']), axis=1)
        comparison_df_with_summary['similarity_score_difference_of_means'] = comparison_df.apply(
            lambda row: (float(row['similarity_score_1_new']) + float(row['similarity_score_2_new']) + float(row['similarity_score_3_new']))/3 - 
            (float(row['similarity_score_1_baseline']) + float(row['similarity_score_2_baseline']) + float(row['similarity_score_3_baseline']))/3, axis=1)
        comparison_df_with_summary.to_csv(f"./{test_folder}/comparison_results.{dynamic_filename_part}.csv", index=False)
        print(f"Comparison results saved to {filename}")
        display(comparison_df_with_summary)

else:
    result = rag_pipeline.run_pipeline(
    "how is alzheimers diagnosed?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
    )

    print(f"Response: {result['response']}")
    print(f"\nContext Count: {result['context_count']}")
    print(f"Similarity Scores: {result['similarity_scores']}")


/Users/dereky/old/personal/code/ai-makerspace-code/AIE9/02_Dense_Vector_Retrieval
Response: The provided context does not include specific information about the early warning signs of heart disease or explicit guidance on when to see a doctor for these symptoms. The information mainly addresses risk factors for coronary heart disease (CHD), the importance of diet and lifestyle changes to reduce risk, and general recommendations for heart-healthy behaviors.

For example, the sources emphasize the importance of a healthy diet and lifestyle in reducing the risk of heart disease and related conditions, such as high blood cholesterol, high blood pressure, and obesity (Source 1). They also discuss controllable and uncontrollable risk factors for CHD (Source 2) and provide dietary recommendations to avoid unhealthy fats and sodium (Source 3). However, none of these sources specify the early warning signs or symptoms to watch for, nor do they provide clear instructions on when to seek medical 

,question_number,question,response,context_count,similarity_score_1,similarity_score_2,similarity_score_3
0,1,What are the early warning signs of heart dise...,The provided context does not include specific...,3,1.029,1.040,1.065
1,2,What lifestyle changes can help manage Type 2 ...,"To manage Type 2 diabetes without medication, ...",3,0.854,0.953,0.960
2,3,How is Alzheimer's disease diagnosed and what ...,Alzheimer's disease (AD) is diagnosed through ...,3,0.973,1.046,1.047
3,4,What are the most effective treatments for chr...,The most effective treatments for chronic obst...,3,0.770,0.827,0.875
4,5,What screening tests are recommended for early...,The recommended screening tests for early dete...,3,0.828,0.887,0.965
5,6,How can chronic kidney disease be slowed or pr...,"Chronic kidney disease (CKD) cannot be cured, ...",3,0.846,0.983,1.053
6,7,What are the warning signs of a stroke and wha...,The warning signs of a stroke often happen qui...,3,0.815,0.854,0.878
7,8,What medications are commonly prescribed for m...,"Based on the provided context, medicines such ...",3,0.947,0.995,1.060
8,9,How is arthritis diagnosed and what treatments...,Arthritis is diagnosed by a health care provid...,3,0.900,0.956,0.985
9,10,What are the best approaches for treating and ...,The best approaches for treating and managing ...,3,0.897,1.025,1.045


Results saved to results.experiment_20260118_144625.csv
Comparison results saved to results.experiment_20260118_144625.csv


,question_number,question_baseline,response_baseline,context_count_baseline,similarity_score_1_baseline,similarity_score_2_baseline,similarity_score_3_baseline,question_new,response_new,context_count_new,similarity_score_1_new,similarity_score_2_new,similarity_score_3_new,summary,similarity_score_1_diff,similarity_score_2_diff,similarity_score_3_diff,similarity_score_difference_of_means
0,1,What are the early warning signs of heart dise...,I don't have information about the early warni...,3,0.471,0.459,0.433,What are the early warning signs of heart dise...,The provided context does not include specific...,3,1.029,1.040,1.065,Change,0.558,0.581,0.632,0.590333
1,2,What lifestyle changes can help manage Type 2 ...,Based on the information provided in the conte...,3,0.636,0.546,0.540,What lifestyle changes can help manage Type 2 ...,"To manage Type 2 diabetes without medication, ...",3,0.854,0.953,0.960,Change,0.218,0.407,0.420,0.348333
2,3,How is Alzheimer's disease diagnosed and what ...,Alzheimer's disease (AD) is diagnosed through ...,3,0.527,0.453,0.452,How is Alzheimer's disease diagnosed and what ...,Alzheimer's disease (AD) is diagnosed through ...,3,0.973,1.046,1.047,Change,0.446,0.593,0.595,0.544667
3,4,What are the most effective treatments for chr...,The most effective treatments for chronic obst...,3,0.704,0.658,0.617,What are the most effective treatments for chr...,The most effective treatments for chronic obst...,3,0.770,0.827,0.875,Change,0.066,0.169,0.258,0.164333
4,5,What screening tests are recommended for early...,The recommended screening tests for early dete...,3,0.657,0.607,0.535,What screening tests are recommended for early...,The recommended screening tests for early dete...,3,0.828,0.887,0.965,Change,0.171,0.280,0.430,0.293667
5,6,How can chronic kidney disease be slowed or pr...,"Chronic kidney disease (CKD) cannot be cured, ...",3,0.642,0.517,0.445,How can chronic kidney disease be slowed or pr...,"Chronic kidney disease (CKD) cannot be cured, ...",3,0.846,0.983,1.053,Change,0.204,0.466,0.608,0.426000
6,7,What are the warning signs of a stroke and wha...,The warning signs of a stroke often happen qui...,3,0.668,0.635,0.615,What are the warning signs of a stroke and wha...,The warning signs of a stroke often happen qui...,3,0.815,0.854,0.878,Change,0.147,0.219,0.263,0.209667
7,8,What medications are commonly prescribed for m...,"Based on the provided information, medications...",3,0.552,0.505,0.438,What medications are commonly prescribed for m...,"Based on the provided context, medicines such ...",3,0.947,0.995,1.060,Change,0.395,0.490,0.622,0.502333
8,9,How is arthritis diagnosed and what treatments...,Arthritis is diagnosed through a combination o...,3,0.595,0.542,0.514,How is arthritis diagnosed and what treatments...,Arthritis is diagnosed by a health care provid...,3,0.900,0.956,0.985,Change,0.305,0.414,0.471,0.396667
9,10,What are the best approaches for treating and ...,The best approaches for treating and managing ...,3,0.598,0.475,0.454,What are the best approaches for treating and ...,The best approaches for treating and managing ...,3,0.897,1.025,1.045,Change,0.299,0.550,0.591,0.480000


In [64]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True,
    distance_measure_string=distance_measure_string
)

questionCounter = 1
questionArray = [
    "What are the early warning signs of heart disease and when should I see a doctor?",
    "What lifestyle changes can help manage Type 2 diabetes without medication?",
    "How is Alzheimer's disease diagnosed and what are the current treatment options?",
    "What are the most effective treatments for chronic obstructive pulmonary disease (COPD)?",
    "What screening tests are recommended for early detection of colorectal cancer?",
    "How can chronic kidney disease be slowed or prevented from progressing?",
    "What are the warning signs of a stroke and what should I do if someone is having one?",
    "What medications are commonly prescribed for managing clinical depression?",
    "How is arthritis diagnosed and what treatments help reduce joint inflammation?",
    "What are the best approaches for treating and managing obesity long-term?",
    "what is the best treatment for Celiac disease?",
    "what are the common symptoms of Lupus?"
]
for question in questionArray:
    result = rag_pipeline.run_pipeline(
        question,
        k=3,
        response_length="comprehensive", 
        include_warnings=True,
        confidence_required=True
    )
    print(f"Response: {result['response']}")
    print(f"\nContext Count: {result['context_count']}")
    print(f"Similarity Scores: {result['similarity_scores']}")
    print(f"Question {questionCounter} of {len(questionArray)}")
    print("--------------------------------")
    questionCounter += 1

Response: I don't have information about the early warning signs of heart disease and when to see a doctor in the provided context. The sources mainly focus on diet, risk factors, and lifestyle recommendations related to heart disease but do not specify early warning signs or guidance on when to seek medical attention.

For accurate diagnosis and advice about heart disease symptoms and the appropriate timing to consult a doctor, it is important to speak with a healthcare professional. They can provide personalized guidance based on your health status.

Context Count: 3
Similarity Scores: ['Source 1: 1.029', 'Source 2: 1.040', 'Source 3: 1.065']
Question 1 of 12
--------------------------------
Response: Based on the provided context, lifestyle changes that can help manage type 2 diabetes without medication include following a healthy eating plan and getting regular physical activity. Managing type 2 diabetes involves balancing what you eat and drink with your physical activity to maint

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
What prompting strategies could you use?
* we're doing zero shot so you could use one-shot or few-shot prompts to give the LLM examples of how to respond, those would contain more detail and examples of appropriate responses.
* there is also a reasoning {"effort" : "low"} parameter you could increase.
* Role based prompting is in play here as well, you could instruct the LLM in the system prompt they are a pHD level person and to provide maximum insight and detail.


### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

 # Pythonic RAG Homework - Experiment Documentation

## Overview

This notebook explores improvements to a RAG (Retrieval Augmented Generation) system through systematic experimentation with embedding models and distance metrics. All experiments include a test harness for reproducible evaluation.

---

## Addition 1: Expanded Domain Data

**Goal:** Broaden the corpus for better evaluation coverage

### Changes Made
- Added medical data covering the **top 12 diseases in the US**
- Each disease includes: treatment options, symptoms, and risk factors
- Increases diversity in test scenarios for more meaningful evaluation

---

## Addition 2: Test Harness

**Goal:** Enable systematic, reproducible performance evaluation

### How It Works
- Runs 12 standardized user prompts through the RAG pipeline
- Records results to CSV files for comparison
- Captures similarity scores for direct evaluation of retrieval quality

### Configuration Flags

| Flag | Default | Purpose |
|------|---------|---------|
| `QA_MODE` | `True` | Run all 12 prompts instead of single query |
| `COMPARE_TO_BASELINE` | `True` | Generate comparison analysis |
| `PERFORM_NEW_RUN` | `True` | Generate new results (vs. static compare) |
| `BASELINE_FILE` | `results.experiment_baseline.csv` | Reference baseline |
| `COMPARE_TO_FILE` | `results.experiment_baseline.csv` | Comparison target |

### Output Files (in `/test/` directory)

| File | Description |
|------|-------------|
| `results.experiment_baseline.csv` | Baseline before algorithmic changes |
| `comparison_results.experiment_baseline_to_baseline.csv` | Self-test confirming no diffs |

---

## Addition 3: Embedding Model Comparison

**Goal:** Evaluate different embedding models on retrieval quality

### Experiment Setup
- **Baseline:** `text-embedding-3-small`
- **Test:** `text-embedding-ada-002`
- **Results:** `comparison_results.experiment_20260118_144222.csv`

### Key Findings

#### 1. Ada-002 Produces Consistently Higher Scores
- Every question shows positive difference (0.21 to 0.36)
- Ada-002 scores are **21-36% higher** than text-embedding-3-small

#### 2. Score Distribution Differs Significantly

| Model | Score Range | Characteristic |
|-------|-------------|----------------|
| `text-embedding-3-small` | 0.43 - 0.71 | Spread out |
| `text-embedding-ada-002` | 0.81 - 0.91 | Compressed, high |

**Interpretation:** Ada-002 produces vectors that are "closer together" in embedding space. This is a *calibration difference*, not necessarily a quality difference.

#### 3. All Responses Changed
Every row shows "Change" — different documents were retrieved or ranked differently.

### Analysis: What Do Higher Scores Mean?

⚠️ **Higher Scores ≠ Better Retrieval**

The higher scores from Ada-002 could indicate:
- **More semantic clustering** — similar concepts embed closer together
- **Less discrimination** — harder to distinguish somewhat-related from highly-related content
- **Different training objectives** — models were trained differently

### Better Evaluation Approach

Instead of raw similarity scores, compare:
- Are Ada-002 responses more **accurate**?
- Do they cite **better sources**?
- Are there questions where one model clearly **wins**?

**Example — Question 1 (Heart Disease):**
- Both models acknowledge missing early warning sign data
- Ada-002 response provides **more depth and context**
- Ada-002 appears to perform slightly better here

### Future Work
- [ ] Test `text-embedding-3-large` vs baseline
- [ ] Compare against Ada-002 results
- [ ] Monitor OpenAI credit usage

---

## Addition 4: Distance Metric Comparison

**Goal:** Evaluate alternative distance metrics for retrieval

### Implementation
- Reset embedding to `text-embedding-3-small`
- Added multiple distance measure functions:


python
distance_measure_options = [
"cosine_similarity",
"euclidean_distance",
"manhattan_distance",
"chebyshev_distance"
]


- Refactored to pass string through pipeline, resolve to callable at execution
- Handled inverted sort order for cosine vs. other metrics

### Experiment: Cosine Similarity vs. Euclidean Distance
- **Results:** `comparison_results.experiment_20260118_144625.csv`

### Key Finding

⚠️ **Comparison is not meaningful!**

- Euclidean distance is **not normalized** (0-1) like cosine similarity
- Raw scores can't be directly compared
- Math validates correctly, but metrics measure different things

### Distance Metrics Reference

| Metric | Best For | RAG Applicability |
|--------|----------|-------------------|
| **Cosine Similarity** | Semantic similarity (text) | ✅ **Recommended** |
| **Euclidean** | Spatial/geometric data | ❌ Magnitude-dependent |
| **Manhattan** | Sparse, high-dimensional data | ❌ Not better than cosine |
| **Chebyshev** | Worst-case dimension mismatch | ❌ Rarely useful for text |

### Why Cosine Similarity Wins for RAG

1. **Ignores Magnitude, Focuses on Direction**
   - Text embeddings encode *meaning* as direction
   - Vector lengths don't matter — only where they point

2. **Normalized Comparison**
   - Always returns 0-1 range (easy to interpret)
   - Not affected by document length variations

3. **Model Optimization**
   - OpenAI, Cohere, etc. optimize for cosine similarity
   - Other metrics may produce unexpected results

### Practical Recommendation

For this health/wellness RAG system:
1. **Stick with Cosine Similarity** — it's what embedding models expect
2. **Focus on higher-impact improvements:**
   - Better chunking strategies
   - Hybrid search (semantic + keyword)
   - Reranking retrieved results
   - Improved prompts

### Key Insight 💡

> The distance metric is rarely the bottleneck in RAG quality. These typically matter more:
> - Chunking strategy
> - Embedding model choice *(already tested!)*
> - Number of retrieved chunks (k)
> - Context formatting for the LLM

---

## Summary of Experiments

| Experiment | Finding | Recommendation |
|------------|---------|----------------|
| Ada-002 vs Small | Ada-002 scores 21-36% higher, more depth in responses | Consider Ada-002 for production |
| Euclidean vs Cosine | Metrics not directly comparable | Use Cosine Similarity |
| Manhattan/Chebyshev | Not advantageous for semantic text | Use Cosine Similarity |

## Next Steps
- [ ] Test `text-embedding-3-large`
- [ ] Experiment with chunking strategies
- [ ] Evaluate hybrid search approaches


In [65]:
"""
ORIGINAL RESPONSE:
Response: Based on the provided context, some natural remedies for improving sleep quality include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a therapeutic method specifically for managing insomnia (Source 1).

2. Relaxation techniques such as progressive muscle relaxation and meditation (Sources 1 and 3).

3. Herbal teas like chamomile or valerian root can promote relaxation and better sleep (Source 1).

4. Magnesium supplements may help, though it is advised to consult a healthcare provider before use (Source 1).

Additionally, improving sleep hygiene practices can further enhance sleep quality, including:
- Avoiding caffeine after 2 PM,
- Exercising regularly but not close to bedtime,
- Limiting alcohol and heavy meals before bed,
- Maintaining a consistent sleep and wake schedule,
- Creating a relaxing bedtime routine,
- Keeping the bedroom temperature between 65-68°F (18-20°C),
- Using blackout curtains or a sleep mask,
- Keeping the bedroom quiet (possibly with white noise) and comfortable with quality mattresses and pillows (Sources 1, 2, and 3).

These combined natural approaches and environment adjustments can support better sleep quality.

Please remember to consult with healthcare professionals for personalized medical advice or before starting supplements or new therapies.

Context Count: 3
Similarity Scores: ['Source 1: 0.553', 'Source 2: 0.521', 'Source 3: 0.439']

"""

"\nORIGINAL RESPONSE:\nResponse: Based on the provided context, some natural remedies for improving sleep quality include:\n\n1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a therapeutic method specifically for managing insomnia (Source 1).\n\n2. Relaxation techniques such as progressive muscle relaxation and meditation (Sources 1 and 3).\n\n3. Herbal teas like chamomile or valerian root can promote relaxation and better sleep (Source 1).\n\n4. Magnesium supplements may help, though it is advised to consult a healthcare provider before use (Source 1).\n\nAdditionally, improving sleep hygiene practices can further enhance sleep quality, including:\n- Avoiding caffeine after 2 PM,\n- Exercising regularly but not close to bedtime,\n- Limiting alcohol and heavy meals before bed,\n- Maintaining a consistent sleep and wake schedule,\n- Creating a relaxing bedtime routine,\n- Keeping the bedroom temperature between 65-68°F (18-20°C),\n- Using blackout curtains or a sleep mask,\n- Keep